## For testing GEFS/R2 import and plotting - climatology data


In [1]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

from herbie import Herbie

In [2]:
H = Herbie(
    "2017-03-14", model="gefs_reforecast", fxx=12, member=0, variable_level="tmp_2m"
)
H.download(verbose=True)

✅ Found ┊ model=gefs_reforecast ┊ product=GEFSv12/reforecast ┊ 2017-Mar-14 00:00 UTC F12 ┊ GRIB2 @ aws ┊ IDX @ aws
👨🏻‍🏭 Created directory: [/Users/johnlawson/data/gefs_reforecast/20170314]
✅ Success! Downloaded GEFS_REFORECAST from aws                 
	src: /Users/johnlawson/data/gefs_reforecast/20170314/tmp_2m_2017031400_c00.grib2
	dst: /Users/johnlawson/data/gefs_reforecast/20170314/tmp_2m_2017031400_c00.grib2


PosixPath('/Users/johnlawson/data/gefs_reforecast/20170314/tmp_2m_2017031400_c00.grib2')

In [3]:
H.inventory().search_this


0     :TMP:2 m above ground:3 hour fcst:ENS=low-res ctl
1     :TMP:2 m above ground:6 hour fcst:ENS=low-res ctl
2     :TMP:2 m above ground:9 hour fcst:ENS=low-res ctl
3     :TMP:2 m above ground:12 hour fcst:ENS=low-res...
4     :TMP:2 m above ground:15 hour fcst:ENS=low-res...
                            ...                        
75    :TMP:2 m above ground:228 hour fcst:ENS=low-re...
76    :TMP:2 m above ground:231 hour fcst:ENS=low-re...
77    :TMP:2 m above ground:234 hour fcst:ENS=low-re...
78    :TMP:2 m above ground:237 hour fcst:ENS=low-re...
79    :TMP:2 m above ground:240 hour fcst:ENS=low-re...
Name: search_this, Length: 80, dtype: object

In [4]:
ds = H.xarray(":15 hour fcst:")
ds


<xarray.Dataset>
Dimensions:              (latitude: 721, longitude: 1440)
Coordinates:
    number               int64 0
    time                 datetime64[ns] 2017-03-14
    step                 timedelta64[ns] 15:00:00
    heightAboveGround    float64 2.0
  * latitude             (latitude) float64 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude            (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    valid_time           datetime64[ns] 2017-03-14T15:00:00
Data variables:
    t2m                  (latitude, longitude) float32 242.8 242.8 ... 225.7
    gribfile_projection  object None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   gefs_reforecast
    product:                 GEFSv12/reforecast
    description:             Global Ensemble Forecast System (GEFS)
    remote_grib:             /Users/johnlawson/data/gefs_reforecast/20170314/...
    local_grib:              /Users/johnlawson/data/gefs_reforecast/20170314/...
    searchString:            :15 hour fcst:

In [5]:
import s3fs
import pandas as pd
# Now for looking up levels:

In [6]:
# List files in the GEFS bucket for a day
fs = s3fs.S3FileSystem(anon=True)
files = fs.ls(
    path="noaa-gefs-retrospective/GEFSv12/reforecast/2015/2015010100/c00/Days:1-10"
)

In [7]:
# var_lev prefix
var_lev = [i.split("/")[-1].split("_") for i in files if i.endswith(".grib2")]
variable_levels_df = pd.DataFrame(var_lev, columns=["variable", "level", "a", "b", "c"])
variable_levels_df

,variable,level,a,b,c
0,acpcp,sfc,2015010100,c00.grib2,None
1,apcp,sfc,2015010100,c00.grib2,None
2,cape,sfc,2015010100,c00.grib2,None
3,cin,sfc,2015010100,c00.grib2,None
4,dlwrf,sfc,2015010100,c00.grib2,None
...,...,...,...,...,...
56,vgrd,pvor,2015010100,c00.grib2,None
57,vvel,pres,2015010100,c00.grib2,None
58,vvel,pres,abv700mb,2015010100,c00.grib2
59,watr,sfc,2015010100,c00.grib2,None


In [8]:
# These are the available variables
variable_levels_df.variable.unique()

array(['acpcp', 'apcp', 'cape', 'cin', 'dlwrf', 'dswrf', 'gflux', 'gust',
       'hgt', 'hlcy', 'lhtfl', 'ncpcp', 'pbl', 'pres', 'pvort', 'pwat',
       'rh', 'sfcr', 'shtfl', 'soilw', 'spfh', 'tcdc', 'tmax', 'tmin',
       'tmp', 'tozne', 'tsoil', 'uflx', 'ugrd', 'ulwrf', 'uswrf', 'vflx',
       'vgrd', 'vvel', 'watr', 'weasd'], dtype=object)

In [9]:
# These are the available levels
variable_levels_df.level.unique()

array(['sfc', 'ceiling', 'hybr', 'pres', 'hgt', 'msl', 'mslet', 'pvor',
       'isen', 'eatm', 'bgrnd', '2m', 'tatm'], dtype=object)